# 1. Adding dropout and normalization layers
Study the pytorch documentation for:
- Dropout https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html
- normalization layers https://pytorch.org/docs/stable/nn.html#normalization-layers

Experiment with adding dropout and normalization layers to your model. Some rough guidelines where to add them relative to Linear or Conv2d layers:
- Dropout: after Linear or Conv2d layers. Often added after the last Linear layer *before* the output layer, but could occur more often.
- Normalization layers: right after (blocks of) Linear or Conv2d layers, but before activation functions.

In [1]:
from pathlib import Path
import torch
import torch.nn as nn
from loguru import logger
import warnings
warnings.simplefilter("ignore", UserWarning)

In [2]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import BasePreprocessor

for dataset in DatasetType:
    print(dataset)

DatasetType.FLOWERS
DatasetType.IMDB
DatasetType.GESTURES
DatasetType.FASHION
DatasetType.SUNSPOTS
DatasetType.IRIS
DatasetType.PENGUINS
DatasetType.FAVORITA
DatasetType.SECURE


In [3]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor
preprocessor = PaddedPreprocessor()

#fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
#streamers = fashionfactory.create_datastreamer(batchsize=64, preprocessor=preprocessor)
gesturesfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

2024-12-13 12:13:58.357 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\Francesca\.cache\mads_datasets\gestures
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 651/651 [00:00<00:00, 1534.55it/s]


In [43]:
len(train), len(valid)

(81, 20)

In [4]:
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))
x.shape, y.shape

(torch.Size([32, 33, 3]), torch.Size([32]))

In [70]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the 1D CNN model
class Gesture1DCNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(Gesture1DCNN, self).__init__()

        # First convolutional layer
        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)

        # # Second convolutional layer
        # self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        # self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        # # Third convolutional layer (optional)
        # self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        # self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)

        # Fully connected layers
        # The sequence length will reduce by a factor of 8 after 3 pooling layers
        self.fc1 = nn.Linear(256 * 3, 128)  # 256 channels, sequence length = 3 after pooling
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        # Change the shape of the input tensor to (batch_size, num_channels, sequence_length)
        x = x.permute(0, 2, 1)  # Now the shape is (batch_size, 3, 30)
        
        # Apply first convolutional layer, relu, and max pooling
        x = self.pool1(torch.relu(self.conv1(x)))
        print(x.shape)
        # # Apply second convolutional layer, relu, and max pooling
        # x = self.pool2(torch.relu(self.conv2(x)))
        # print(x.shape)
        # # Apply third convolutional layer, relu, and max pooling
        # x = self.pool3(torch.relu(self.conv3(x)))
        # print(x.shape)
        
        # Flatten the output from convolutional layers to feed into fully connected layers
        x = x.view(x.size(0), -1)  # Flatten: (batch_size, 256 * 3)
        print(x.shape)
        # Apply fully connected layers
        x = torch.relu(self.fc1(x))  
        x = self.fc2(x)  # Output layer (no activation here, will apply softmax during loss computation)
        
        return x


In [47]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the 1D CNN model with one convolutional layer
class Gesture1DCNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(Gesture1DCNN, self).__init__()

        # Single convolutional layer
        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=64, kernel_size=5, stride=1, padding=2)  # kernel size 5, stride 1, padding 2 for same length
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)  # MaxPool with kernel size 2 and stride 2

        # # Second convolutional layer
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        # Third convolutional layer (optional)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        # Global Max Pooling Layer 
        self.global_pool = nn.AdaptiveMaxPool1d(1)  # Global max pooling reduces each feature map to a single value

        # Fully connected layer
        self.fc1 = nn.Linear(256, 128)  
        self.fc2 = nn.Linear(128, 64)  
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        # Change the shape of the input tensor to (batch_size, num_channels, sequence_length)
        x = x.permute(0, 2, 1)  # Now the shape is (batch_size, 3, 30)
        
        # Apply the convolutional layer, relu activation, and max pooling
        x = self.pool1(torch.relu(self.conv1(x)))  # (batch_size, 64, 15) after convolution and pooling

        x = self.pool2(torch.relu(self.conv2(x)))  # (batch_size, 64, 15) after convolution and pooling

        x = self.pool3(torch.relu(self.conv3(x)))  # (batch_size, 64, 15) after convolution and pooling
        
        # Apply Global Max Pooling: reduces each feature map to a single value
        x = self.global_pool(x)  # (batch_size, 64, 1)

        # Remove the last dimension (sequence length is 1) for fully connected layer
        x = x.view(x.size(0), -1)  # Flatten to (batch_size, 64)
        
        # Apply fully connected layers
        x = torch.relu(self.fc1(x))  
        x = torch.relu(self.fc2(x))  
        x = self.fc3(x)  # Output layer (no activation here, will apply softmax during loss computation)
        
        return x



In [48]:

input_channels = 3  # Number of input channels (e.g., x, y, z accelerometer data)
num_classes = 20     # Number of gesture classes

# Instantiate the model
model = Gesture1DCNN(input_channels=input_channels, num_classes=num_classes)

# Print model summary
print(model)

output = model(x)
print(output.shape)  # Should be (32, num_classes) — batch size x number of classes


Gesture1DCNN(
  (conv1): Conv1d(3, 64, kernel_size=(5,), stride=(1,), padding=(2,))
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (global_pool): AdaptiveMaxPool1d(output_size=1)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=20, bias=True)
)
torch.Size([32, 20])


In [49]:
import torch.optim as optim
from mltrainer import metrics, Trainer
optimizer = optim.Adam
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = metrics.Accuracy()

In [50]:
log_dir = Path("../../models/cnn").resolve()
if not log_dir.exists():
    log_dir.mkdir(parents=True)

In [68]:
from mltrainer import TrainerSettings, ReportTypes

settings = TrainerSettings(
    epochs=5,
    metrics=[accuracy],
    logdir=log_dir,
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
)
settings

epochs: 5
metrics: [Accuracy]
logdir: C:\Users\Francesca\Documents\osint\code_repo\AI\MADS-MachineLearning-FP\dev\models\cnn
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TENSORBOARD: 2>, <ReportTypes.MLFLOW: 3>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.1, 'patience': 10}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 10}

In [73]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import mlflow
import torch.optim as optim
from mltrainer import metrics, Trainer, TrainerSettings, ReportTypes
from datetime import datetime
experiment_path = "mlflow_test"
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import BasePreprocessor
from loguru import logger

In [74]:
experiment_path = "mlflow_cnn1D-gestures"

In [75]:
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

using cuda


In [76]:
trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optimizer,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
    device=device,
    )

2024-12-13 13:43:17.382 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to C:\Users\Francesca\Documents\osint\code_repo\AI\MADS-MachineLearning-FP\dev\models\cnn\20241213-134317
2024-12-13 13:43:17.386 | INFO     | mltrainer.trainer:__init__:72 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.


In [65]:
#trainer.loop()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 128.46it/s]
2024-12-13 13:15:08.441 | INFO     | mltrainer.trainer:report:191 - Epoch 0 train 0.0078 test 0.0676 metric ['0.9891']
100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 244.37it/s]
2024-12-13 13:15:08.812 | INFO     | mltrainer.trainer:report:191 - Epoch 1 train 0.0373 test 0.1223 metric ['0.9734']
2024-12-13 13:15:08.813 | INFO     | mltrainer.trainer:__call__:234 - best loss: 0.0676, current loss 0.1223.Counter 1/10.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 246.15it/s]
2024-12-13 13:15:09.184 | INFO     | mltrainer.trainer:report:191 - Epoch 2 train 0.0214 test 0.0829 metric ['0.9844']
2024-12-13 13:15:09.185 | INFO     | mltrainer.trainer:__call__:234 - best loss: 0.0676, current loss 0.08

# Use MLFLOW
Start mlflow with:

```
mlflow server     --backend-store-uri sqlite:///mlflow.db     --default-artifact-root ./mlruns     --host 127.0.0.1:5000
```

In [77]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import mlflow
from datetime import datetime
import torch.optim as optim
from mltrainer import metrics, Trainer, TrainerSettings, ReportTypes
from datetime import datetime
experiment_path = "mlflow_test"
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import BasePreprocessor
from loguru import logger

In [ ]:


mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("gestures_1Dconv")
modeldir = Path("../../models/gestures/").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)

#gin.parse_config_file("gestures_gru.gin")
model = Gesture1DCNN()

with mlflow.start_run():
    mlflow.set_tag("model", "Gesture1DCNN")
    mlflow.set_tag("dev", "francesca")
    mlflow.log_params(gin.get_bindings("Gesture1DCNN"))

    model = rnn_models.GRUmodel()

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau,
        device=device,
    )
    trainer.loop()

    tag = datetime.now().strftime("%Y%m%d-%H%M")
    modelpath = modeldir / (tag + "model.pt")
    torch.save(model, modelpath)